## Merging Weather & Flight Data - plus additional cleanup

### Imports

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

### User-set Parameters

In [2]:
weather_data_dir = 'cleaned_output'
flight_data_dir = 'filtered_data'

In [3]:
timezone_data = pd.read_csv('airport_timezone.csv')

In [4]:
#pd.read_csv('filtered_data/filtered_2018').info()

### Helper Functions

In [5]:
def rounddown(x):
    #round a number down to the nearest hundredth -do this to convert HHmm to nearest hour before
    if (int(x) % 100 == 0):
        return int(x)
    else:
        return (int(x) - int(x) % 100)

def split_datetime(df):
    #for a given weather dataframe, convert the DATE column to include only the date,
    #and create a new TIME column to include the time formatted HHmm
    date_col = [df['DATE'][i].split(' ')[0] for i in range(len(df['DATE']))]
    time_col = [df['DATE'][i].split(' ')[1][:-3].replace(':','') for i in range(len(df['DATE']))]
    df['DATE'] = date_col
    df['TIME'] = time_col

def convert_date(date_col):
    #convert string date into date object
    converted_date_col = [datetime.strptime(x, '%Y-%m-%d').date() for x in date_col]
    return converted_date_col
    
def hour_col(df):
    #for a given dataframe, create a new hour column from each time column. 
    #This will be used to join weather & flight data by nearest hour
    if 'CRSDepTime' in df.columns:
        df['CRSDepHr'] = [rounddown(x) for x in df['CRSDepTime']]
    if 'CRSArrTime' in df.columns:
        df['CRSArrHr'] = [rounddown(x) for x in df['CRSArrTime']]
    if 'TIME' in df.columns:
        df['HOUR'] = [rounddown(x) for x in df['TIME']]
        
def days_near_last_hol(df):
    #takes in flight dataframe, adds columns for the number of days until and since the last (US) holiday
    #holidays considered: Halloween, US Thanksgiving, Christmas Eve, Christmas, New Year's Eve, New Year's Day, MLK Jr Day
    #0 means: day is a holiday. Halloween considered so Nov 1st isn't treated as holiday, and 2024 Valentine's day is considered as an upper cap
    thanksgiving_dates = ['2018-11-22', '2019-11-28','2020-11-26','2021-11-25','2022-11-24','2023-11-23']
    mlk_dates = ['2019-01-21','2020-01-20','2021-01-18','2022-01-17','2023-01-16','2024-01-15']
    halloween_dates = [str(y) + '-10-31' for y in range(2018,2024)]
    xmas_eve_dates = [str(y) + '-12-24' for y in range(2018,2024)]
    xmas_day_dates = [str(y) + '-12-25' for y in range(2018,2024)]
    nye_dates = [str(y) + '-12-31' for y in range(2018, 2024)]
    nyd_dates = [str(y) + '-01-01' for y in range(2019,2025)]
    val_day_2024_date = ['2024-02-14']
    holidays = [halloween_dates, thanksgiving_dates, mlk_dates, xmas_eve_dates, xmas_day_dates, nye_dates, nyd_dates,val_day_2024_date]
    hol_dates = [x for xs in holidays for x in xs] #inspired by https://stackoverflow.com/a/952952\
    #convert from str to datetime.date format
    flightdates = convert_date(df['FlightDate'])
    hol_dates = convert_date(hol_dates)
    #create days until holiday column: min(holiday - flightdate), where holiday is not before flightdate
    days_until = [min([(hdate - x).days for hdate in hol_dates if hdate >= x]) for x in flightdates] 
    #create days since holiday column: min(flightdate - holiday), where holiday is not after flightdate
    days_after = [min([(x - hdate).days for hdate in hol_dates if hdate <= x]) for x in flightdates]
    df['DaysUntilHol'] = days_until
    df['DaysAfterHol'] = days_after
    
def convert_timezone(df, airport_type):
    for i in range(len(df['Time_Zone'])):
        time_change = 0 #will remain 0 for CST locations
        
        if df['Time_Zone'][i] == 'EST':
            time_change = -100
        elif df['Time_Zone'][i] == 'MST':
            time_change = 100
        elif df['Time_Zone'][i] == 'PST':
            time_change = 200
        elif df['Time_Zone'][i] == 'AKST':
            time_change = 300
        elif df['Time_Zone'][i] == 'HAST':
            time_change = 400

        if airport_type == 'Origin':
            df.loc[i,'CRSDepTime'] == df['CRSDepTime'][i] + time_change
            df.loc[i,'DepTime'] == df['DepTime'][i] + time_change
        else: #airport_type == 'Dest'
            df.loc[i,'CRSArrTime'] == df['CRSArrTime'][i] + time_change
            df.loc[i,'ArrTime'] == df['ArrTime'][i] + time_change
    print('finished processing for ' + airport_type)
    #special handling for when converted CRS time(s) alter FlightDate?
            
            
def convert_to_cst(flight_season_data, timezone_data):
    #convert time to cst for all flight data csvs
    #if converting time will change date (e.g. roll back 1 hour will push date back to previous day), update FlightDate too
    with_origin_tz = pd.merge(
        left = flight_season_data,
        right = timezone_data,
        how = 'inner',
        left_on = flight_season_data['Origin'],
        right_on = timezone_data['Airport_Code'])
    convert_timezone(with_origin_tz, 'Origin')
    flight_season_data = with_origin_tz.drop(columns=['Airport_Code','Time_Zone','key_0'])
    with_dest_tz = pd.merge(
        left = flight_season_data,
        right = timezone_data,
        how = 'inner',
        left_on = flight_season_data['Dest'],
        right_on = timezone_data['Airport_Code'])
    convert_timezone(with_dest_tz, 'Dest')
    flight_season_data = with_dest_tz.drop(columns=['Airport_Code','Time_Zone','key_0'])
    
    print('finished converting times')
    return flight_season_data
    

### Consolidation & Merging Functions

In [6]:
def consolidate_weather(weather_data_dir):
    #pull each airport weather csv, add airport code & hour cols
    #consolidate all airports' weather data & write to separate csv
    weather_dfs = []
    
    for file in os.listdir(weather_data_dir): 
        if '.ipynb' not in file and 'weather' not in file: #avoid trying to parse existing consolidation
            airport_code = file[:-4] #remove the '.csv' to get just airport code
            file_df = pd.read_csv(os.path.join(weather_data_dir, file))
            file_df['AIRPORT_CODE'] = airport_code
            split_datetime(file_df)
            hour_col(file_df)
            weather_dfs.append(file_df)
        
    weather_data = pd.concat(weather_dfs)
    weather_data.to_csv(os.path.join(weather_data_dir, 'weather.csv'), index=False)
    print('weather data consolidation complete!')
    
    
def unify_weather_flight(flight_data_dir, weather_data_dir):
    weather_data = pd.read_csv(os.path.join(weather_data_dir, 'weather.csv'))
    flight_weather_dfs = []
    
    for file in os.listdir(flight_data_dir):
        flight_season_data = pd.read_csv(os.path.join(flight_data_dir, file))
        days_near_last_hol(flight_season_data) #get holiday closeness columns
        flight_season_data = convert_to_cst(flight_season_data,timezone_data)
        hour_col(flight_season_data) #need hour cols to match hourly weather rows
        #first merge to get arrival airport weather:
        with_arr_weather = pd.merge(
            left = flight_season_data, 
            right = weather_data, 
            how = 'inner', 
            left_on = ['Dest', 'FlightDate','CRSDepHr'],
            right_on = ['AIRPORT_CODE','DATE','HOUR'])
        #then merge again to get departure airport weather:
        flight_season_with_weather = pd.merge(
            left = with_arr_weather,
            right = weather_data,
            how = 'inner',
            left_on = ['Origin', 'FlightDate','CRSDepHr'],
            right_on = ['AIRPORT_CODE','DATE','HOUR'],
            suffixes = ('_DEST','_ORIGIN')) #suffix to distinguish weather data
        #drop extraneous columns:
        flight_season_with_weather.drop(columns=['DATE_DEST','DATE_ORIGIN','AIRPORT_CODE_DEST','AIRPORT_CODE_ORIGIN','HOUR_DEST','HOUR_ORIGIN'])
        flight_weather_dfs.append(flight_season_with_weather)
        print('processed ' + file)
    
    flight_weather_data = pd.concat(flight_weather_dfs)
    flight_weather_data.to_csv('flight_weather.csv', index=False)
    print('data unification complete!')

In [7]:
#consolidate_weather(weather_data_dir)

In [8]:
unify_weather_flight(flight_data_dir, weather_data_dir)

C:\Users\armer\AppData\Local\Temp\ipykernel_18156\1962729451.py:21: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_data = pd.read_csv(os.path.join(weather_data_dir, 'weather.csv'))


finished processing for Origin
finished processing for Dest
finished converting times
processed filtered_2018.csv


C:\Users\armer\AppData\Local\Temp\ipykernel_18156\1962729451.py:25: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  flight_season_data = pd.read_csv(os.path.join(flight_data_dir, file))


finished processing for Origin
finished processing for Dest
finished converting times
processed filtered_2019.csv
finished processing for Origin
finished processing for Dest
finished converting times
processed filtered_2021.csv
finished processing for Origin
finished processing for Dest
finished converting times
processed filtered_2022.csv
finished processing for Origin
finished processing for Dest
finished converting times
processed filtered_2023.csv
data unification complete!


In [9]:
all_data = pd.read_csv('flight_weather.csv')

C:\Users\armer\AppData\Local\Temp\ipykernel_18156\494517200.py:1: DtypeWarning: Columns (36,38,39,56,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('flight_weather.csv')


In [10]:
test_set = all_data.head(10000)

In [12]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 66 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   DayOfWeek                         10000 non-null  int64  
 1   FlightDate                        10000 non-null  object 
 2   Marketing_Airline_Network         10000 non-null  object 
 3   Origin                            10000 non-null  object 
 4   Dest                              10000 non-null  object 
 5   CRSDepTime                        10000 non-null  int64  
 6   DepTime                           9856 non-null   float64
 7   DepDelay                          9856 non-null   float64
 8   CRSArrTime                        10000 non-null  int64  
 9   ArrTime                           9848 non-null   float64
 10  ArrDelay                          9825 non-null   float64
 11  Cancelled                         10000 non-null  int64  
 12  Cance

In [11]:
test_set[['DayOfWeek','FlightDate','DaysUntilHol','DaysAfterHol']][200:2000]

,DayOfWeek,FlightDate,DaysUntilHol,DaysAfterHol
200,4,2018-11-15,7,15
201,4,2018-11-15,7,15
202,4,2018-11-15,7,15
203,4,2018-11-15,7,15
204,4,2018-11-15,7,15
...,...,...,...,...
1995,7,2018-11-25,29,3
1996,7,2018-11-25,29,3
1997,7,2018-11-25,29,3
1998,7,2018-11-25,29,3
